## Data Validation with TensorFlow Data Validation

---

In [1]:
# Import libraries

import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv

from __future__ import print_function

In [2]:
# Import dataset

dataset = pd.read_csv("/Users/mareksturek/Documents/GitHub/deep-learning/data/pollution-small.csv")
dataset.head()

,Date,pm10,no2,so2,soot
0,1/1/2009,98.67,14.10,44.38,34.81
1,1/2/2009,52.33,14.10,29.75,33.06
2,1/3/2009,74.67,20.50,36.25,39.25
3,1/4/2009,72.00,17.30,46.44,34.38
4,1/5/2009,81.00,25.64,56.56,45.59


In [3]:
# Dataset shape

dataset.shape

(2188, 5)

In [4]:
# Tranining data

training_data = dataset[:1600]

In [5]:
# Training data summary

training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [6]:
# Test data

test_set = dataset[1600:]

In [7]:
# Test data summary

test_set.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


In [8]:
# Data analysis and validation with TFDV

# Generate training data statistics

train_stats = tfdv.generate_statistics_from_dataframe(dataframe=dataset)

In [9]:
# Infering the schema

schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


In [10]:
# Calculate test set statistics

test_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set)

In [11]:
# Compare test statistics with the Schema

# Checking for anomalies in new data

anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)

In [12]:
# Displaying all detected anomalies

'''
- Integer larger than 10
- STRING type when expected INT type
- FLOAT type when expected INT type
- Integer smaller than 0
'''

tfdv.display_anomalies(anomalies)

In [13]:
# New data WITH anomalies

test_set_copy = test_set.copy()
test_set_copy.drop("soot", axis=1, inplace=True)

In [14]:
# Statistics based on data with anomalies

test_set_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set_copy)
anomalies_new = tfdv.validate_statistics(statistics=test_set_copy_stats, schema=schema)
tfdv.display_anomalies(anomalies_new)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:186: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


In [15]:
# Prepare the schema for Serving

schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

In [16]:
# Removing a target column from the Serving shema

tfdv.get_feature(schema, "soot").not_in_environment.append("SERVING")

In [17]:
# Checking for anomalies between the SERVING environment and new test set

serving_env_anomalies = tfdv.validate_statistics(test_set_copy_stats, schema, environment="SERVING")
tfdv.display_anomalies(serving_env_anomalies)

In [19]:
# Freezing the schema

tfdv.write_schema_text(schema=schema, output_path="pollution_schema.pbtxt")